In [5]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get('https://www.azlyrics.com/r/rem.html', headers=headers)
doc = BeautifulSoup(response.text)

In [7]:
rows = []
albums = doc.find(id="listAlbum")
songs = albums.find_all('a')
for song in songs:
    row = {}
    row['title'] = song.text
    row['slug'] = song['href']
    rows.append(row)

df = pd.DataFrame(rows)

In [8]:
df.head()

,slug,title
0,../lyrics/rem/radiofreeeurope.html,Radio Free Europe
1,../lyrics/rem/pilgrimage.html,Pilgrimage
2,../lyrics/rem/laughing.html,Laughing
3,../lyrics/rem/talkaboutthepassion.html,Talk About The Passion
4,../lyrics/rem/moralkiosk.html,Moral Kiosk


In [12]:
df['slug'] = df.slug.str.extract(r"..(.*)")

In [20]:
df.tail()

,slug,title
203,/lyrics/rem/mikespopsong.html,Mike's Pop Song
204,/lyrics/rem/photograph.html,Photograph
205,/lyrics/rem/romance.html,Romance
206,/lyrics/rem/thegreatbeyond.html,The Great Beyond
207,/lyrics/rem/weallgobacktowherewebelong.html,We All Go Back To Where We Belong


In [15]:
df.to_csv("rem_songs.csv",index=False)
df = pd.read_csv("rem_songs.csv")

In [16]:
url = "https://www.azlyrics.com/lyrics/rem/catapult.html"
print("We are scraping", url)
response = requests.get(url)
doc = BeautifulSoup(response.text)

# sorry, i know its not sexy but it just seemed like the easiest option. plus its wokring... :D
# I double checked with 5 songs :)

doc.find(class_="ringtone").next_element.next_element.next_element.next_element.next_element\
.next_element.next_element.next_element.next_element.next_element.next_element.text.strip() 

We are scraping https://www.azlyrics.com/lyrics/rem/catapult.html


"We were little boys, we were little girls\nIt's nine o'clock, don't try to turn it off\nCowered in a hole, ope your mouth\n\nDid we miss anything, did we miss anything\nDid we miss anything, did we miss anything\nCatapult (catapult), catapult\nCatapult (catapult), catapult\n\nWe were little boys, we were little girls\nIt's nine o'clock, don't try to turn it off\nCowered in a hole, ope your mouth\nWe in step, in hand, your mother remembers this\nHear the howl of the rope, a question\n\nDid we miss anything, did we miss anything\nDid we miss anything, did we miss anything\nCatapult (catapult), catapult\nCatapult (catapult), catapult\n\nMarch could be darker, March could be darker\nCatapult (catapult), catapult\nCatapult (catapult), catapult\n\nWe were little boys, we were little girls\nIt's nine o'clock, don't try to turn it off\nCowered in a hole, open your mouth\nWe in step, in hand, your mother remembers this\nHear the howl of the rope, a question\n\nDid we miss anything, did we miss

In [17]:
def scrape_page(row):
    url = f"https://www.azlyrics.com{row['slug']}"
    print("Scraping", row['slug'])

    response = requests.get(url)
    doc = BeautifulSoup(response.text)

    page = {}

    # sorry, i know its not sexy but it just seemed like the easiest option. plus its wokring... :D
    # I double checked with 5 songs :)
    
    page['lyrics'] = doc.find(class_="ringtone").next_element.next_element.next_element.next_element.next_element\
    .next_element.next_element.next_element.next_element.next_element.next_element.text.strip() 

    return pd.Series(page)

In [18]:
scraped_df = df.apply(scrape_page, axis=1)

Scraping /lyrics/rem/radiofreeeurope.html
Scraping /lyrics/rem/radiofreeeurope.html
Scraping /lyrics/rem/pilgrimage.html
Scraping /lyrics/rem/laughing.html
Scraping /lyrics/rem/talkaboutthepassion.html
Scraping /lyrics/rem/moralkiosk.html
Scraping /lyrics/rem/perfectcircle.html
Scraping /lyrics/rem/catapult.html
Scraping /lyrics/rem/sittingstill.html
Scraping /lyrics/rem/99.html
Scraping /lyrics/rem/shakingthrough.html
Scraping /lyrics/rem/wewalk.html
Scraping /lyrics/rem/westofthefields.html
Scraping /lyrics/rem/harborcoat.html
Scraping /lyrics/rem/sevenchinesebrothers.html
Scraping /lyrics/rem/socentralrain.html
Scraping /lyrics/rem/prettypersuasion.html
Scraping /lyrics/rem/timeaftertimeannelise.html
Scraping /lyrics/rem/secondguessing.html
Scraping /lyrics/rem/letterneversent.html
Scraping /lyrics/rem/camera.html
Scraping /lyrics/rem/dontgobacktorockville.html
Scraping /lyrics/rem/littleamerica.html
Scraping /lyrics/rem/windout.html
Scraping /lyrics/rem/moonriver.html
Scraping /lyr

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), 'occurred at index 133')

In [19]:
scraped_df.head()

NameError: name 'scraped_df' is not defined

In [ ]:
final_df = df.merge(scraped_df, left_index=True, right_index=True)

In [ ]:
final_df.to_csv("rem_lyrics.csv",index=False)